In [1]:
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import torch
from time import time
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    AutoModelForSequenceClassification,
    Trainer,
    EarlyStoppingCallback
)
from trl import SFTTrainer,setup_chat_format
import numpy as np
from transformers import BitsAndBytesConfig, AutoModelForSequenceClassification
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoTokenizer
from datasets import DatasetDict, Dataset

In [ ]:
import pandas as pd
import numpy as np

# Define the path to your parquet file
parquet_path = '/home/xiaoyicheng/pjlab/LMSYS/dataset/train.parquet'

# Read the parquet file
train = pd.read_parquet(parquet_path)

# 不进行采样，直接使用全量数据
train.reset_index(drop=True, inplace=True)

def process(input_str):
    """
    Process the input string by removing surrounding brackets,
    splitting by '","', stripping quotes, and joining the sentences.
    """
    if pd.isna(input_str):
        return ''
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return ' '.join(sentences)

# Apply the processing function to 'prompt', 'response_a', and 'response_b' columns
train['prompt'] = train['prompt'].apply(process)
train['response_a'] = train['response_a'].apply(process)
train['response_b'] = train['response_b'].apply(process)

# 删除 'response_a' 和 'response_b' 都为 'null' 或空字符串的行
null_responses = (
    (train['response_a'].str.lower() == 'null') & 
    (train['response_b'].str.lower() == 'null')
) | (
    (train['response_a'] == '') & 
    (train['response_b'] == '')
)
indexes_to_drop = train[null_responses].index
train.drop(indexes_to_drop, inplace=True)
train.reset_index(drop=True, inplace=True)

print(f"Total {len(indexes_to_drop)} Null response rows dropped")
print('Total train samples:', len(train))

# Map the 'winner' column to numerical labels
# Assuming 'winner' can be 'a', 'b', or 'tie'
label_mapping = {'a': 0, 'b': 1, 'tie': 2}
train['label'] = train['winner'].map(label_mapping)

# Handle any unexpected or missing values in 'winner'
if train['label'].isnull().any():
    print("Warning: Some 'winner' values were not recognized and will be set to -1")
    train['label'].fillna(-1, inplace=True)
    train['label'] = train['label'].astype(int)

# Combine 'prompt', 'response_a', and 'response_b' into a single 'text' column
train['text'] = (
    'User prompt: ' + train['prompt'] +
    '\n\nModel A:\n' + train['response_a'] +
    '\n\n--------\n\nModel B:\n' + train['response_b']
)

# Display a sample from the 'text' column
print(train['text'].iloc[4])

# (Optional) Save the processed DataFrame for future use
# train.to_csv('processed_train.csv', index=False)
# or
# train.to_parquet('processed_train.parquet', index=False)


Total 0 Null response rows dropped
Total train samples: 484
User prompt: You are the leadership team of Anderson University (Indiana). A small-sized, private, faith-based (nondenominational Christian) institution facing several challenges and opportunities. The University is experiencing declining enrollment, increased competition from online courses, and financial constraints. However, there are also opportunities for growth.

What can you plan that will:

Increase enrollment by 15% within the next three years.
Improve the university's financial stability.
Enhance the quality of education and student experience.
Strengthen the university's reputation and brand

Model A:
To address the challenges and leverage the opportunities for Anderson University, we need a comprehensive strategic plan that focuses on growth, financial sustainability, educational quality, and brand strengthening. Here’s a detailed plan:

### 1. Increase Enrollment by 15% within Three Years

#### a. Diversified Mark

/tmp/ipykernel_55008/2189808328.py:47: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['label'].fillna(-1, inplace=True)
